In [2]:
from pathlib import Path
import pandas as pd
import sqlite3
from src.utils.os_helper import get_project_root
import os

repo = get_project_root()

In [ ]:
CACHE_DIR = repo / "cache" / "data" / "awesome_cgm" / "aleppo"
data_tables = CACHE_DIR / "raw" / "Data Tables"
db_path = CACHE_DIR / "awesome_cgm.db"

# SQLite param cap (commonly 999). Keep a margin.
SQLITE_MAX_VARS = 999
MARGIN = 10

In [14]:
con = sqlite3.connect(db_path)
cur = con.cursor()
cur.execute("PRAGMA journal_mode=WAL;")
cur.execute("PRAGMA synchronous=OFF;")
cur.execute("PRAGMA temp_store=MEMORY;")

for f in sorted(data_tables.glob("*.txt")):
    table = f.stem
    print(f"Importing {f.name} -> {table}")

    # Stream in chunks to control memory and SQL variables
    first = True
    for df in pd.read_csv(f, sep="|", dtype=str, low_memory=False, chunksize=50_000):
        ncols = len(df.columns)
        # rows per insert so (rows * cols) <= SQLITE_MAX_VARS - MARGIN
        safe_rows = max(1, (SQLITE_MAX_VARS - MARGIN) // max(1, ncols))

        # method=None avoids multi-row SQL text construction; Pandas will executemany
        df.to_sql(
            table,
            con,
            if_exists="replace" if first else "append",
            index=False,
            chunksize=safe_rows,
            method=None,
        )
        first = False


indexes = [
    # Bolus Indexes
    "CREATE INDEX idx_hdevicebolus_ptid ON HDeviceBolus(PtID);",
    "CREATE INDEX idx_hdevicebolus_parentid ON HDeviceBolus(ParentHDeviceUploadsID);",
    "CREATE INDEX idx_hdevicebolus_ptid_days ON HDeviceBolus(PtID, DeviceDtTmDaysFromEnroll);",
    "CREATE INDEX idx_hdevicebolus_normal ON HDeviceBolus(Normal) WHERE Normal IS NOT NULL;",
    "CREATE INDEX idx_hdevicebolus_order ON HDeviceBolus(PtID, DeviceDtTmDaysFromEnroll, DeviceTm);",
    # Uploads Indexes
    "CREATE INDEX idx_hdeviceuploads_recid ON HDeviceUploads(RecID);",
    "CREATE INDEX idx_hdeviceuploads_ptid ON HDeviceUploads(PtID);",
    "CREATE INDEX idx_hdeviceuploads_device ON HDeviceUploads(DeviceModel, DeviceType);",
    # CGM Indexes
    "CREATE INDEX idx_hdevicecgm_recid ON HDeviceCGM(RecID);",
    "CREATE INDEX idx_hdevicecgm_recordtype ON HDeviceCGM(RecordType);",
    "CREATE INDEX idx_hdevicecgm_ptid_days ON HDeviceCGM(PtID, DeviceDtTmDaysFromEnroll);",
    "CREATE INDEX idx_hdevicecgm_ptid_time ON HDeviceCGM(PtID, DeviceTm);",
    # Wizard Indexes
    "CREATE INDEX idx_hdevicewizard_recid ON HDeviceWizard(RecID);",
    "CREATE INDEX idx_hdevicewizard_ptid ON HDeviceWizard(PtID);",
    "CREATE INDEX idx_hdevicewizard_ptid_days ON HDeviceWizard(PtID, DeviceDtTmDaysFromEnroll);",
    "CREATE INDEX idx_hdevicewizard_ptid_time ON HDeviceWizard(PtID, DeviceTm);",
]

for i, index_sql in enumerate(indexes):
    try:
        cur.execute(index_sql)
        print(
            f"Created index {i+1}/{len(indexes)}: {index_sql.split('ON ')[1].split('(')[0]}"
        )
    except sqlite3.Error as e:
        print(f"Error creating index: {e}")

con.commit()
con.close()

Importing HAEDeviceProblems.txt -> HAEDeviceProblems
Importing HAdvEvent.txt -> HAdvEvent
Importing HComplUnblindCGM.txt -> HComplUnblindCGM
Importing HDeviceBGM.txt -> HDeviceBGM
Importing HDeviceBasal.txt -> HDeviceBasal
Importing HDeviceBolus.txt -> HDeviceBolus
Importing HDeviceCGM.txt -> HDeviceCGM
Importing HDeviceDtTmVer.txt -> HDeviceDtTmVer
Importing HDeviceEvents.txt -> HDeviceEvents
Importing HDeviceIssue.txt -> HDeviceIssue
Importing HDeviceUploads.txt -> HDeviceUploads
Importing HDeviceWizard.txt -> HDeviceWizard
Importing HFollowUp.txt -> HFollowUp
Importing HHypoEvent.txt -> HHypoEvent
Importing HInitialStudyCGM.txt -> HInitialStudyCGM
Importing HInsulin.txt -> HInsulin
Importing HLocalHbA1c.txt -> HLocalHbA1c
Importing HMedicalCondition.txt -> HMedicalCondition
Importing HMedication.txt -> HMedication
Importing HPostRandPtFinalStatus.txt -> HPostRandPtFinalStatus
Importing HPtRoster.txt -> HPtRoster
Importing HQuestDiabTech.txt -> HQuestDiabTech
Importing HQuestHypoFear

In [3]:
def query_db(query: str):
    # Open the connection first
    con = sqlite3.connect(db_path)
    # cur = con.cursor()
    df = pd.read_sql_query(query, con)
    con.close()
    return df

### Convert data to csv

- pid
- date
- tableType
- bolusType
- normalBolus
- expectedNormalBolus
- extendedBolus
- expectedExtendedBolus
- bgInput
- foodG
- iob
- cr
- isf
- bgMgdl
- rate
- suprBasalType
- suprRate

In [ ]:
query = """
WITH
  params AS (
    SELECT '2020-01-01' AS base_date
  )
SELECT
    pid, date, tableType,
    bolusType, normalBolus, expectedNormalBolus, extendedBolus, expectedExtendedBolus,
    bgInput, foodG, iob, cr, isf,
    bgMgdl,
    rate, basalDurationMins, suprBasalType, suprRate
FROM (
    -- Bolus data
    SELECT
        HDeviceBolus.PtID as pid,
        datetime(julianday((SELECT base_date FROM params)) + CAST(HDeviceBolus.DeviceDtTmDaysFromEnroll AS INTEGER) + (julianday(HDeviceBolus.DeviceTm) - julianday('00:00:00'))) AS date,
        'bolus' as tableType,
        HDeviceBolus.BolusType as bolusType,
        HDeviceBolus.Normal as normalBolus,
        HDeviceBolus.ExpectedNormal as expectedNormalBolus,
        HDeviceBolus.Extended as extendedBolus,
        HDeviceBolus.ExpectedExtended as expectedExtendedBolus,
        NULL as bgInput,
        NULL as foodG,
        NULL as iob,
        NULL as cr,
        NULL as isf,
        NULL as bgMgdl,
        NULL as rate,
        NULL as basalDurationMins,
        NULL as suprBasalType,
        NULL as suprRate
    FROM HDeviceBolus

    UNION ALL

    -- Wizard data
    SELECT
        HDeviceWizard.PtID as pid,
        datetime(julianday((SELECT base_date FROM params)) + CAST(HDeviceWizard.DeviceDtTmDaysFromEnroll AS INTEGER) + (julianday(HDeviceWizard.DeviceTm) - julianday('00:00:00'))) AS date,
        'wizard' as tableType,
        NULL as bolusType,
        NULL as normalBolus,
        NULL as expectedNormalBolus,
        NULL as extendedBolus,
        NULL as expectedExtendedBolus,
        HDeviceWizard.BgInput as bgInput,
        HDeviceWizard.CarbInput as foodG,
        HDeviceWizard.InsulinOnBoard as iob,
        HDeviceWizard.InsulinCarbRatio as cr,
        HDeviceWizard.InsulinSensitivity as isf,
        NULL as bgMgdl,
        NULL as rate,
        NULL as basalDurationMins,
        NULL as suprBasalType,
        NULL as suprRate
    FROM HDeviceWizard

    UNION ALL

    -- CGM data
    SELECT
        HDeviceCGM.PtID as pid,
        datetime(julianday((SELECT base_date FROM params)) + CAST(HDeviceCGM.DeviceDtTmDaysFromEnroll AS INTEGER) + (julianday(HDeviceCGM.DeviceTm) - julianday('00:00:00'))) AS date,
        'cgm' as tableType,
        NULL as bolusType,
        NULL as normalBolus,
        NULL as expectedNormalBolus,
        NULL as extendedBolus,
        NULL as expectedExtendedBolus,
        NULL as bgInput,
        NULL as foodG,
        NULL as iob,
        NULL as cr,
        NULL as isf,
        HDeviceCGM.GlucoseValue as bgMgdl,
        NULL as rate,
        NULL as basalDurationMins,
        NULL as suprBasalType,
        NULL as suprRate
    FROM HDeviceCGM
    WHERE HDeviceCGM.GlucoseValue IS NOT NULL
    AND HDeviceCGM.RecordType = 'CGM'

    UNION ALL

    -- Basal data
    SELECT
        HDeviceBasal.PtID as pid,
        datetime(julianday((SELECT base_date FROM params)) + CAST(HDeviceBasal.DeviceDtTmDaysFromEnroll AS INTEGER) + (julianday(HDeviceBasal.DeviceTm) - julianday('00:00:00'))) AS date,
        'basal' as tableType,
        NULL as bolusType,
        NULL as normalBolus,
        NULL as expectedNormalBolus,
        NULL as extendedBolus,
        NULL as expectedExtendedBolus,
        NULL as bgInput,
        NULL as foodG,
        NULL as iob,
        NULL as cr,
        NULL as isf,
        NULL as bgMgdl,
        HDeviceBasal.Rate as rate,
        HDeviceBasal.Duration / 60000 as basalDurationMins,
        HDeviceBasal.SuprBasalType as suprBasalType,
        HDeviceBasal.SuprRate as suprRate
    FROM HDeviceBasal
)
ORDER BY pid, date ASC;
"""
df_all = query_db(query)
df_all.to_csv("output.csv")

### Raw -> Interim

In [14]:
def convert_to_csv(df):
    project_root = get_project_root()
    data_dir = project_root / "cache" / "data" / "awesome_cgm" / "aleppo" / "interim"
    os.makedirs(data_dir, exist_ok=True)
    for pid in df["pid"].unique():
        df_pid = df[df["pid"] == pid]
        df_pid.to_csv(data_dir / f"p{pid}_full.csv", index=False)
        print(f"Done processing pid {pid}")


convert_to_csv(df_all)

Done processing pid 10
Done processing pid 101
Done processing pid 102
Done processing pid 103
Done processing pid 105
Done processing pid 106
Done processing pid 108
Done processing pid 109
Done processing pid 11
Done processing pid 110
Done processing pid 111
Done processing pid 112
Done processing pid 113
Done processing pid 115
Done processing pid 116
Done processing pid 118
Done processing pid 119
Done processing pid 121
Done processing pid 123
Done processing pid 124
Done processing pid 127
Done processing pid 128
Done processing pid 129
Done processing pid 130
Done processing pid 131
Done processing pid 132
Done processing pid 134
Done processing pid 135
Done processing pid 136
Done processing pid 137
Done processing pid 138
Done processing pid 139
Done processing pid 14
Done processing pid 140
Done processing pid 141
Done processing pid 143
Done processing pid 145
Done processing pid 146
Done processing pid 147
Done processing pid 148
Done processing pid 149
Done processing pid

In [7]:
from src.data.models import ColumnNames
from src.data.preprocessing.pipeline import preprocessing_pipeline
from src.utils.unit import mg_dl_to_mmol_l
import pandas as pd
from src.utils.os_helper import get_project_root

import logging

logger = logging.getLogger(__name__)


# 	pid	date	tableType	bolusType	normalBolus	expectedNormalBolus	extendedBolus	expectedExtendedBolus	bgInput	foodG	iob	cr	isf	bgMgdl	rate	suprBasalType	suprRate
def data_translation(df_raw: pd.DataFrame) -> pd.DataFrame:
    """
    'pid' -> p_num
    'date' -> datetime
    'tableType' -> msg_type (bolus, wizard, cgm). wizard contains information like carbs intake
    'eventType': This is bolus type
    'normal': Number of units of normal bolus
    'expectedNormal'
    'extended': Number of units for extended delivery
    'expectedExtended'
    'bgInput' -> Blood glucose as inputted into wizard in mg
    'carbInput' -> Carbohydrates as inputted into wizard in mg
    'iob': Units of insulin on board
    'cr': Number of mg carbs covered by unit of insulin. Not used yet but we can find a way to give model a hint about the slope of the glucose curve
    'isf': Number of bgs covered by unit of insulin. Same as above
    'recordType': CGM | CALIBRATION
    'glucoseValue' -> bg_mM
    """
    df = df_raw.copy()
    # TODO: Rename to the correct column names
    df = df.rename(
        columns={
            "pid": ColumnNames.P_NUM.value,
            "date": ColumnNames.DATETIME.value,
            "tableType": ColumnNames.MSG_TYPE.value,
            "normalBolus": ColumnNames.DOSE_UNITS.value,
            # "expectedNormalBolus": ColumnNames.EXPECTED_NORMAL.value,
            # "extendedBolus": ColumnNames.EXTENDED.value,
            # "expectedExtendedBolus": ColumnNames.EXPECTED_EXTENDED.value,
            # "bgInput": ColumnNames.BG.value, todo: Maybe tag the record type as bgInput
            "foodG": ColumnNames.FOOD_G.value,
            "iob": ColumnNames.IOB.value,
            # "cr": ColumnNames.INSULIN_CARB_RATIO.value,
            # "isf": ColumnNames.ISF.value,
            "recordType": ColumnNames.RECORD_TYPE.value,
            "bgMgdl": ColumnNames.BG.value,
            # "rate": ColumnNames.RATE.value,
            "basalDurationMins": ColumnNames.BASAL_DURATION_MINS.value,
            "suprBasalType": ColumnNames.SUPR_BASAL_TYPE.value,
            "suprRate": ColumnNames.SUPR_RATE.value,
        }
    )
    # Drop the expected columns for now
    df.drop(columns=["expectedNormalBolus", "expectedExtendedBolus"], inplace=True)
    df.set_index(ColumnNames.DATETIME.value, inplace=True)
    df.sort_index(inplace=True)
    df.index = pd.to_datetime(df.index)

    # Convert blood glucose from mg/dL to mmol/L
    df[ColumnNames.BG.value] = mg_dl_to_mmol_l(df, bgl_col=ColumnNames.BG.value)

    return df


def keep_overlapping_data(patient_df: pd.DataFrame) -> pd.DataFrame:
    """
    Args:
        patient_df: A dataframe that has been through data_translation (datetime is the index)

    Keep data that has overlapping time windows for all table types.
    Note that not all patients have data for all table types so we only consider the table types that are present.

    Don't think we are losing too much data here. Most patients still have at least 6 months worth of data after this step except for some patients.
    - p216, p019, p081, p289, p138 (check 3.18 notebook for more details)
    """
    table_types = patient_df[ColumnNames.MSG_TYPE.value].unique()
    start_datetime = None  # This should be the max of all the min datetimes
    end_datetime = None  # This should be the min of all the max datetimes

    for table_type in table_types:
        table_data = patient_df[patient_df[ColumnNames.MSG_TYPE.value] == table_type]
        if table_data.empty:
            continue

        min_datetime = table_data.index.min()
        max_datetime = table_data.index.max()

        # Find the latest start time of all table types
        if start_datetime is None or min_datetime > start_datetime:
            start_datetime = min_datetime

        # Find the earliest end time of all table types
        if end_datetime is None or max_datetime < end_datetime:
            end_datetime = max_datetime

    if start_datetime is None or end_datetime is None:
        return None

    has_overlap = start_datetime < end_datetime
    if not has_overlap:
        # This shouldn't happen
        logger.warning(
            f"Patient {patient_df[ColumnNames.P_NUM.value].iloc[0]} has no overlapping data"
        )
        return None

    # Filter using the index (datetime)
    return patient_df[
        (patient_df.index >= start_datetime) & (patient_df.index <= end_datetime)
    ]

In [13]:
from src.data.preprocessing.generic_cleaning import erase_consecutive_nan_values
from src.data.preprocessing.time_processing import get_most_common_time_interval


def process_one_patient(
    df_raw: pd.DataFrame, debug: bool = False, verbose: bool = False
) -> pd.DataFrame:
    """
    Process the raw data for one patient:
        1. Translate the data (columns and units)
        2. Keep overlapping data (for bolus, wizard, cgm and basal)
        3. Rollover basal rate to the next few rows if the rate is not null
        4. preprocessing_pipeline (This include resampling and deriving cob and iob)
    """
    HOURS_OF_CONSECUTIVE_NAN_VALUES = 6
    df = df_raw.copy()

    # Translate the data
    df = data_translation(df)
    pid = df[ColumnNames.P_NUM.value].iloc[0]

    # Keep overlapping data
    logger.info(f"Keeping overlapping data for patient {pid}")
    df = keep_overlapping_data(df)
    if df is None:
        return None

    # Print data meta: span of datetime index and number of rows
    if verbose:
        start_dt = df.index.min()
        end_dt = df.index.max()
        food_g = df[df[ColumnNames.FOOD_G.value].notna()]
        bolus = df[df[ColumnNames.DOSE_UNITS.value].notna()]
        logger.info(
            f"Patient {pid} processed data spans from {start_dt} to {end_dt} ({(end_dt - start_dt)})"
        )
        logger.info(f"Number of rows with food intake: {len(food_g)}")
        logger.info(f"Number of rows with bolus: {len(bolus)}")

    # Resampling to constant interval, rollover basal rate and derive cob and iob
    df = preprocessing_pipeline(pid, df, use_aggregation=True)

    # Drop days with more than 6 hours of consecutive NaN values
    freq_mins = get_most_common_time_interval(df)
    max_consecutive_nan_values_per_day = (
        HOURS_OF_CONSECUTIVE_NAN_VALUES * 60
    ) // freq_mins
    # Note that it is possible that we have bolus from the deleted days. becase we rollover first then delete.
    df = erase_consecutive_nan_values(
        df, max_consecutive_nan_values_per_day=max_consecutive_nan_values_per_day
    )

    # Debug only
    if debug:
        debug_dir = (
            get_project_root() / "cache" / "data" / "awesome_cgm" / "aleppo" / "debug"
        )
        os.makedirs(debug_dir, exist_ok=True)
        df.to_csv(debug_dir / f"p{pid}.csv", index=True)
    return df


def process_all_patients(
    interim_path: Path,
    processed_path: Path,
) -> pd.DataFrame:
    """
    Clean all patients' data in the interim path and save the processed data to the processed path.
    """
    total_patients = len(os.listdir(interim_path))
    os.makedirs(processed_path, exist_ok=True)
    for index, pid in enumerate(os.listdir(interim_path)):
        progress = f"({index+1}/{total_patients})"
        save_path = processed_path / pid

        # Don't process the patient if the processed data already exists
        if save_path.exists():
            print(f"Skipping pid {pid} because {save_path} already exists {progress}.")
            continue

        df = pd.read_csv(interim_path / pid)
        df = process_one_patient(df)
        df.to_csv(save_path, index=True)
        print(f"{"-"*10}Done processing pid {pid} {progress} {"-"*10}")
    return

### Interim -> Processed for one patient

In [9]:
from src.utils.os_helper import get_project_root
import pandas as pd

PID = 3

root = get_project_root()
patient_df = pd.read_csv(
    f"{root}/cache/data/awesome_cgm/aleppo/interim/p{PID}_full.csv"
)

processed_df = process_one_patient(patient_df)
processed_df.to_csv(f"p{PID}_full.csv", index=True)

2025-11-03T21:49:17 - Keeping overlapping data for patient 3
2025-11-03T21:49:17 - ==============================
2025-11-03T21:49:17 - Preprocessing patient 3
2025-11-03T21:49:17 - ==============================
2025-11-03T21:49:17 - create_physiological_features(): Deriving features...
2025-11-03T21:49:17 - 	Ensuring regular time intervals with aggregation...
2025-11-03T21:49:17 - ensure_regular_time_intervals_with_aggregation(): Ensuring regular time intervals with aggregation...
2025-11-03T21:49:17 - 	Most common time interval: 5 minutes
2025-11-03T21:49:17 - 	Aggregation strategy: {'p_num': 'first', 'msg_type': 'first', 'bolusType': 'first', 'dose_units': 'sum', 'extendedBolus': 'sum', 'bgInput': 'sum', 'food_g': 'sum', 'iob': 'first', 'cr': 'first', 'isf': 'first', 'bg_mM': 'mean', 'rate': 'mean', 'basal_duration_mins': 'sum', 'supr_basal_type': 'first', 'supr_rate': 'sum'}
2025-11-03T21:49:17 - Post-ensure_regular_time_intervals_with_aggregation(): 
			Patient 3 
			 - old index

In [13]:
cgm_df = patient_df[patient_df["tableType"] == "cgm"]
cgm_start = cgm_df.date.min()
cgm_end = cgm_df.date.max()
print(f"CGM data spans from {cgm_start} to {cgm_end}")


bolus_df = patient_df[patient_df["tableType"] == "bolus"]
bolus_start = bolus_df.date.min()
bolus_end = bolus_df.date.max()
print(f"Bolus data spans from {bolus_start} to {bolus_end}")

wizard_df = patient_df[patient_df["tableType"] == "wizard"]
wizard_start = wizard_df.date.min()
wizard_end = wizard_df.date.max()
print(f"Wizard data spans from {wizard_start} to {wizard_end}")

basal_df = patient_df[patient_df["tableType"] == "basal"]
basal_start = basal_df.date.min()
basal_end = basal_df.date.max()
print(f"Basal data spans from {basal_start} to {basal_end}")


data_start = max(cgm_start, bolus_start, wizard_start, basal_start)
data_end = min(cgm_end, bolus_end, wizard_end, basal_end)
print(f"Data spans from {data_start} to {data_end}")


cgm_df.to_csv("cgm_p7.csv")

CGM data spans from 2019-08-17 14:32:00 to 2020-09-30 10:01:44
Bolus data spans from 2019-11-01 20:38:27 to 2020-09-30 08:58:17
Wizard data spans from 2019-11-02 09:19:27 to 2020-09-30 08:58:17
Basal data spans from 2019-11-01 23:12:23 to 2020-09-30 10:14:30
Data spans from 2019-11-02 09:19:27 to 2020-09-30 08:58:17


### Process all patients

In [ ]:
from src.data.diabetes_datasets.awesome_cgm.aleppo.data_cleaner import (
    clean_all_patients,
)
from src.utils.os_helper import get_project_root

repo = get_project_root()

interim_path = repo / "cache" / "data" / "awesome_cgm" / "aleppo" / "interim"
processed_path = repo / "cache" / "data" / "awesome_cgm" / "aleppo" / "processed"

clean_all_patients(interim_path, processed_path)

### use_cached False

In [2]:
from src.data.diabetes_datasets.data_loader import get_loader

loader = get_loader(
    data_source_name="aleppo",
    # dataset_type="train",
    train_percentage=0.9,
    use_cached=False,
    parallel=True,
    max_workers=10,
)

2025-11-04T06:21:52 - Raw data for aleppo already exists in cache
2025-11-04T06:21:52 - Cleaning all patients from /Users/tonychan/GlucoseML/nocturnal-hypo-gly-prob-forecast/cache/data/awesome_cgm/aleppo/interim to /Users/tonychan/GlucoseML/nocturnal-hypo-gly-prob-forecast/cache/data/awesome_cgm/aleppo/processed with parallel=True and max_workers=10
2025-11-04T06:21:52 - Processing 226 patients in parallel with 10 workers
2025-11-04T06:21:54 - Keeping overlapping data for patient 260
2025-11-04T06:21:54 - Keeping overlapping data for patient 47
2025-11-04T06:21:54 - ==============================
2025-11-04T06:21:54 - Preprocessing patient 260
2025-11-04T06:21:54 - ==============================
2025-11-04T06:21:54 - create_physiological_features(): Deriving features...
2025-11-04T06:21:54 - 	Ensuring regular time intervals with aggregation...
2025-11-04T06:21:54 - ensure_regular_time_intervals_with_aggregation(): Ensuring regular time intervals with aggregation...
2025-11-04T06:21:54 